In [1]:
# Run setup_imports to enable importing from src/
%run ../src/setup_imports.py

In [2]:
from src.utils.ibm import get_backend, get_backend_snapshot, run_and_wrap
from src.utils.python_utils import clean_datetimes
from src.utils.circuits import packed_chsh_circuit, phi_plus_meassured, compute_parallel_CHSH_scores
from qiskit_ibm_runtime import SamplerV2 as Sampler
from datetime import datetime
import json
import time
import pandas as pd

/Volumes/X-Wing/casco/Development/introspect-execute-time-drift-experiment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Parametrize experiment
reps = 32
delay_between_reps = 60 * 15 # 15 minutes
introspection_shots = 1024
main_circuit_shots = 1024
entanglement_threshold = 1.9

In [4]:
backend = get_backend()

In [ ]:
# Main experiment loop

results = []

i = 0
while i < reps:
    i+=1
    entry = {}
    
    # --- Backend snapshot; need we do this more than once?---
    entry['backend_snapshot'] = get_backend_snapshot(backend)

    # --- Introspection phase ---
    print(f"{i}. Running introspection circuit")
    entry['introspection'] = run_and_wrap(packed_chsh_circuit(), backend, introspection_shots)
    entry['entanglement_score'] = compute_parallel_CHSH_scores(entry['introspection']["counts"])['CHSH_score']

    # --- Decision phase ---
    if entry['entanglement_score'] > entanglement_threshold:  
        print(f"{i}. Running main circuit - entanglement score was {entry['entanglement_score']}")
        entry['execution'] = run_and_wrap(phi_plus_meassured(), backend, main_circuit_shots)
    else:
        print(f"{i}. Skipping main circuit - entanglement score was {entry['entanglement_score']}")
    
    results.append(clean_datetimes(entry))

    if i < reps:
        time.sleep(delay_between_reps) # wait before next repetition 


1. Running introspection circuit


/Volumes/X-Wing/casco/Development/introspect-execute-time-drift-experiment/.venv/lib/python3.9/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:1111: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


1. Running main circuit - entanglement score was 2.587890625
2. Running introspection circuit
2. Running main circuit - entanglement score was 2.56640625
3. Running introspection circuit
3. Running main circuit - entanglement score was 2.5859375
4. Running introspection circuit
4. Running main circuit - entanglement score was 2.5390625
5. Running introspection circuit
5. Running main circuit - entanglement score was 2.5625
6. Running introspection circuit
6. Running main circuit - entanglement score was 2.548828125
7. Running introspection circuit
7. Running main circuit - entanglement score was 2.560546875
8. Running introspection circuit
8. Running main circuit - entanglement score was 2.515625
9. Running introspection circuit
9. Running main circuit - entanglement score was 2.439453125
10. Running introspection circuit
10. Running main circuit - entanglement score was 2.509765625
11. Running introspection circuit
11. Running main circuit - entanglement score was 2.466796875
12. Runn

In [ ]:
import os
# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Generate a timestamped filename
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
json_filename = f"results/experiment_results_{timestamp}.json"

with open(json_filename, "w") as f:
    json.dump(results, f, indent=2)

# Tabular summary

csv_filename = f"results/experiment_summary_{timestamp}.csv"

df = pd.DataFrame([{
    'introspection_created': r.get('introspection', {}).get('timestamps', {}).get('created'),
    'introspection_running': r.get('introspection', {}).get('timestamps', {}).get('running'),
    'introspection_finished': r.get('introspection', {}).get('timestamps', {}).get('finished'),
    'execution_created': r.get('execution', {}).get('timestamps', {}).get('created'),
    'execution_running': r.get('execution', {}).get('timestamps', {}).get('running'),
    'execution_finished': r.get('execution', {}).get('timestamps', {}).get('finished'),    
    'entanglement_score': r.get('entanglement_score'),
    'execution_counts': json.dumps(r.get('execution', {}).get('counts')) if r.get('execution') else None
} for r in results])
df.to_csv(csv_filename, index=False)